In [137]:
import json
import plotly
import pandas as pd
import joblib
import numpy as np
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from flask import Flask
from flask import render_template, request, jsonify
from plotly.graph_objs import Bar
from sqlalchemy import create_engine

In [138]:
def tokenize(text):
    #norm_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [139]:
# load data
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)

# load model
model = joblib.load("models/classifier.pkl")

In [140]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
df.shape

(26216, 39)

In [142]:
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)

In [143]:
genre_counts

genre
direct    10766
news      13054
social     2396
Name: message, dtype: int64

In [144]:
genre_names

['direct', 'news', 'social']

In [145]:
numeric_df = df.drop(['id', 'message', 'original', 'genre'], axis=1).sum().reset_index()
numeric_df.head()

,index,0
0,related,20094
1,request,4474
2,offer,118
3,aid_related,10860
4,medical_help,2084


In [151]:
numeric_df.columns.values[0] = 'category'
numeric_df.columns.values[1] = 'count'

In [152]:
numeric_df.columns

Index(['category', 'count'], dtype='object')

In [153]:
numeric_df['count'].nlargest(10)

KeyError: 'count'